In [3]:
import cv2
import mediapipe as mp
import pandas as pd
from typing import Tuple
import os

# Hàm tính IoU
def calculate_iou(boxA: Tuple[int, int, int, int], boxB: Tuple[int, int, int, int]) -> float:
    xA1, yA1, xA2, yA2 = boxA
    xB1, yB1, xB2, yB2 = boxB
    
    xI1 = max(xA1, xB1)
    yI1 = max(yA1, yB1)
    xI2 = min(xA2, xB2)
    yI2 = min(yA2, yB2)
    
    inter_width = max(0, xI2 - xI1)
    inter_height = max(0, yI2 - yI1)
    inter_area = inter_width * inter_height
    
    boxA_area = (xA2 - xA1) * (yA2 - yA1)
    boxB_area = (xB2 - xB1) * (yB2 - yB1)
    
    iou = inter_area / float(boxA_area + boxB_area - inter_area)
    
    return iou

def detect_and_draw_face_in_video(video_path, output_folder, video_name):
    """
    Hàm phát hiện khuôn mặt và vẽ bounding box trong video.
    Lưu kết quả vào CSV và video output nếu xử lý video hoàn tất.
    """
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Khởi tạo VideoWriter để lưu video với bounding box
    output_video_path = os.path.join(output_folder, f'{video_name}_output.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (320, 320))

    data = []  # Lưu trữ dữ liệu để lưu vào CSV
    current_boxes = []
    frame_idx = 0
    
    try:

        with mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    print(f"Không thể đọc khung hình từ video: {video_path}")
                    break

                frame_idx += 1
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_detection.process(image_rgb)
                ih, iw, _ = image.shape

                if results.detections:
                    for detection in results.detections:
                        bboxC = detection.location_data.relative_bounding_box
                        x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                        cropped_frame = image[int(y - h/2):int(y + 3*h/2), int(x - h/2):int(x + 3*h/2)]
                        
                        # Resize cropped frame to match the output video size (optional)
                        output_size = (320, 320)
                        cropped_frame_resized = cv2.resize(cropped_frame, output_size)
                    

                        # Thêm bounding box hiện tại vào danh sách
                        current_boxes.append((x, y, x + w, y + h))

                        # Chỉ tính IoU nếu có ít nhất 2 bounding box
                        if len(current_boxes) >= 2:
                            box1, box2 = current_boxes[-2:]
                            iou = calculate_iou(box1, box2)

                            if iou <= 0.5:
                                print(f"IoU giữa hai bounding box gần nhất là {iou}. Dừng xử lý video này.")
                                cap.release()
                                out_video.release()
                                return  # Không lưu CSV nếu dừng giữa chừng

                        # Lưu thông tin bounding box cho từng khung hình
                        data.append([frame_idx, cap.get(cv2.CAP_PROP_POS_MSEC)/1000,  # start_time in seconds
                                    cap.get(cv2.CAP_PROP_POS_MSEC)/1000,  # end_time in seconds
                                    (x, y, x + w, y + h)])

                        # Vẽ bounding box lên video
                        cv2.rectangle(image, (int(x - h/2), int(y - h/2)), (int(x + 3*h/2), int(y + 3*h/2)), (0, 255, 0), 2)
                        cv2.imshow("Detection", image)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break

                # Ghi khung hình đã xử lý vào video output
                # out_video.write(image)
            
                # Write the cropped frame to the output video
                out_video.write(cropped_frame_resized)

        cap.release()
        out_video.release()

        # Chỉ lưu CSV nếu xử lý hoàn tất video
        if data:
            df = pd.DataFrame(data, columns=['frame', 'start_time', 'end_time', 'bbox'])
            output_csv_path = os.path.join(output_folder, f'{video_name}.csv')
            df.to_csv(output_csv_path, index=False)
            print(f"Face tracking complete. Results saved to {output_csv_path}")
    except:
        print('.')

def process_videos_in_folder(input_folder, output_folder):
    """
    Lặp qua tất cả các file .mp4 trong `input_folder` và áp dụng hàm phát hiện khuôn mặt,
    lưu kết quả vào CSV và video output nếu video được xử lý hoàn tất.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.mp4'):
            input_path = os.path.join(input_folder, filename)
            video_name = os.path.splitext(filename)[0]
            print(f"Đang xử lý video: {filename}")
            detect_and_draw_face_in_video(input_path, output_folder, video_name)
            print(f"Hoàn tất xử lý video: {filename}")

if __name__ == "__main__":

    output_folder_video = 'data_output/video'
    # pl_folders = ['KYNAN6', 'LEARN7', 'MOTEP8', 'NGUOI9', 'TEFUN10', 'YEULA11']
    # pl_folders = ['BAIGI6']
    pl_folders = ['KYNAN5']


    # pl_folders = [f.name for f in os.scandir(output_folder_video) if f.is_dir()]
    print(pl_folders)

    for pl_folder in pl_folders:
        if not os.path.exists(output_folder_video + '/' + pl_folder):
            print(f"Error: The folder {input_folder} does not exist.")
        else:
            input_folders = [f.name for f in os.scandir(output_folder_video + '/' + pl_folder) if f.is_dir()]
            for input_folder in input_folders:
                output_folder = os.path.join('data_output/video_non_audio/' + pl_folder, input_folder)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                process_videos_in_folder(output_folder_video + '/' + pl_folder + '/' + input_folder, output_folder)


['KYNAN5']
Đang xử lý video: 3CACH113_1.mp4
.
Hoàn tất xử lý video: 3CACH113_1.mp4
Đang xử lý video: 3CACH113_10.mp4
Không thể đọc khung hình từ video: data_output/video/KYNAN5/3CACH113\3CACH113_10.mp4
Face tracking complete. Results saved to data_output/video_non_audio/KYNAN5\3CACH113\3CACH113_10.csv
Hoàn tất xử lý video: 3CACH113_10.mp4
Đang xử lý video: 3CACH113_100.mp4
Không thể đọc khung hình từ video: data_output/video/KYNAN5/3CACH113\3CACH113_100.mp4
Face tracking complete. Results saved to data_output/video_non_audio/KYNAN5\3CACH113\3CACH113_100.csv
Hoàn tất xử lý video: 3CACH113_100.mp4
Đang xử lý video: 3CACH113_101.mp4
Không thể đọc khung hình từ video: data_output/video/KYNAN5/3CACH113\3CACH113_101.mp4
Face tracking complete. Results saved to data_output/video_non_audio/KYNAN5\3CACH113\3CACH113_101.csv
Hoàn tất xử lý video: 3CACH113_101.mp4
Đang xử lý video: 3CACH113_102.mp4
Không thể đọc khung hình từ video: data_output/video/KYNAN5/3CACH113\3CACH113_102.mp4
Face tracking 